In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
houses_df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [2]:
houses_df['firstandbsmtsf'] = houses_df['totalbsmtsf'] + houses_df['firstflrsf']

import statsmodels.api as sm
X = houses_df[['overallqual','firstandbsmtsf','grlivarea','garagecars','garagearea']]
Y = houses_df['saleprice']
# We need to manually add a constant
# in statsmodels' sm
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     928.7
Date:                Thu, 24 Oct 2019   Prob (F-statistic):               0.00
Time:                        20:15:00   Log-Likelihood:                -17498.
No. Observations:                1460   AIC:                         3.501e+04
Df Residuals:                    1454   BIC:                         3.504e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -1.049e+05   4671.330    -22.460      0.000   -1.14e+05   -9.58e+04
overallqual     2.438e+04   1053.130     23.155      0.000    2.23e+04    2.65e+04
firstandbsmtsf    18.3675      1.673     10.979      0.000      15.086      21.649
grlivarea         41.8914      2.554     16.404      0.000      36.882      46.901
garagecars      1.435e+04   3017.914      4.754      0.000    8427.208    2.03e+04
garagearea        15.8474     10.479      1.512      0.131      -4.708      36.403
==============================================================================
Omnibus:                      415.922   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            35965.590
Skew:                          -0.239   Prob(JB):                         0.00
Kurtosis:                      27.310   Cond. No.                     1.33e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.33e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

The resulting equation:
    
houseprice = -1.049e5 + 2.438e4overallqual + 18.3675firstandbsmtsf + 41.8914grlivarea + 1.435e4garagecars + 15.8474garagearea

With a p value above 0.05, the garagearea variable appears to be insignificant while the rest look statistically significant. We will run the model again and exclude garagearea.

In [3]:
import statsmodels.api as sm
X = houses_df[['overallqual','firstandbsmtsf','grlivarea','garagecars']]
Y = houses_df['saleprice']
# We need to manually add a constant
# in statsmodels' sm
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.761
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     1159.
Date:                Thu, 24 Oct 2019   Prob (F-statistic):               0.00
Time:                        20:18:36   Log-Likelihood:                -17499.
No. Observations:                1460   AIC:                         3.501e+04
Df Residuals:                    1455   BIC:                         3.503e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -1.053e+05   4667.887    -22.550      0.000   -1.14e+05   -9.61e+04
overallqual     2.435e+04   1053.299     23.115      0.000    2.23e+04    2.64e+04
firstandbsmtsf    18.9311      1.632     11.603      0.000      15.731      22.132
grlivarea         42.0556      2.553     16.476      0.000      37.048      47.063
garagecars      1.806e+04   1751.770     10.312      0.000    1.46e+04    2.15e+04
==============================================================================
Omnibus:                      403.165   Durbin-Watson:                   1.977
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            32802.550
Skew:                          -0.181   Prob(JB):                         0.00
Kurtosis:                      26.218   Cond. No.                     1.31e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.31e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

houseprice = -1.053e5 + 2.435e4overallqual + 18.9311firstandbsmtsf + 42.0556grlivarea + 1.806e4garagecars + 15.8474garagearea

The garagecars coefficient experienced a large change, while the remaining coefficients and constant experienced a small change as a result of removing the garagearea variable from the model.The feature that has by far the largest effect on houseprice is the overallqual feature, which makes sense as this is a variable that takes into account a lot of qualities of the houses to come up with an overall score. You would expect this to greatly affect the price of a house by going up 1 point, as it is a ranking out of 10.